In [276]:
import pandas as pd
import scipy.stats as sts
import numpy as np

In [277]:
data = pd.read_csv('botswana.tsv', sep='\t')
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [278]:
len(data["religion"].unique())

4

In [279]:
data.dropna().shape

(1834, 15)

In [280]:
data['nevermarr'] = data["agefm"].apply(lambda x: 1 if np.isnan(x) else 0)

In [281]:
data['nevermarr'].head()

0    1
1    0
2    0
3    1
4    0
Name: nevermarr, dtype: int64

In [282]:
sub_data = data.drop(["evermarr"], axis=1)

In [283]:
sub_data['agefm'].fillna(0, inplace=True)
sub_data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [284]:
sub_data["heduc"][sub_data["nevermarr"]==1] = sub_data["heduc"][sub_data["nevermarr"]==1].fillna(-1)

/home/storm/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [285]:
sub_data["heduc"].isna().sum()

123

In [286]:
for f in ["heduc","idlnchld", "usemeth"]:
    f_noans = f + "_noans"
    sub_data[f_noans] = data[f].apply(lambda x: 1 if np.isnan(x) else 0)

sub_data["heduc"].fillna(-2, inplace=True)
for f in ["idlnchld", "usemeth"]:
    sub_data[f].fillna(-1, inplace=True)
    

In [287]:
sub_data.isnull().sum()

ceb               0
age               0
educ              0
religion          0
idlnchld          0
knowmeth          7
usemeth           0
agefm             0
heduc             0
urban             0
electric          3
radio             2
tv                2
bicycle           3
nevermarr         0
heduc_noans       0
idlnchld_noans    0
usemeth_noans     0
dtype: int64

In [288]:
sub_data.shape[0] * sub_data.shape[1]

78498

In [289]:
sub_data.dropna(inplace=True)

In [290]:
import statsmodels.formula.api as smf
" + ".join(sub_data.columns.tolist())

'ceb + age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + heduc_noans + idlnchld_noans + usemeth_noans'

In [306]:
linear_statmodel = smf.ols("ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + \
                           urban + electric + radio + tv + bicycle + nevermarr + heduc_noans + idlnchld_noans + usemeth_noans", sub_data)

In [307]:
linear_statmodel

In [308]:
fitted_model = linear_statmodel.fit()
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 09 Dec 2018   Prob (F-statistic):               0.00
Time:                        20:18:21   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.212     -4.835      0.000      -1.443      -0.610
religion[T.other]         -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]    -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]        -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                        0.1703      0.003     51.891      0.000       0.164       0.177
educ                      -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                   0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                   0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                    0.6473      0.048     13.424      0.000       0.553       0.742
agefm                     -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                     -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                     -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                  -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                     -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                        -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                    0.2139      0.050      4.260      0.000       0.115       0.312
nevermarr                 -1.3668      0.189     -7.220      0.000      -1.738      -0.996
heduc_noans               -0.8724      0.145     -6.026      0.000      -1.156      -0.589
idlnchld_noans             0.6539      0.153      4.286      0.000       0.355       0.953
usemeth_noans              0.7652      0.196      3.910      0.000       0.382       1.149
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         369.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [294]:
import statsmodels.stats.api as sms
?sms.het_breuschpagan

Signature: sms.het_breuschpagan(resid, exog_het)
Docstring:
Breusch-Pagan Lagrange Multiplier test for heteroscedasticity

The tests the hypothesis that the residual variance does not depend on
the variables in x in the form

:math: \sigma_i = \sigma * f(\alpha_0 + \alpha z_i)

Homoscedasticity implies that $\alpha=0$


Parameters
----------
resid : array-like
    For the Breusch-Pagan test, this should be the residual of a regression.
    If an array is given in exog, then the residuals are calculated by
    the an OLS regression or resid on exog. In this case resid should
    contain the dependent variable. Exog can be the same as x.
    TODO: I dropped the exog option, should I add it back?
exog_het : array_like
    This contains variables that might create data dependent
    heteroscedasticity.

Returns
-------
lm : float
    lagrange multiplier statistic
lm_pvalue :float
    p-value of lagrange multiplier test
fvalue : float
    f-statistic of the hypothesis that the error varianc

In [309]:
sms.het_breuschpagan(fitted_model.resid, fitted_model.model.exog)[1]

1.1452927633439797e-225

In [311]:
test_data = sub_data.drop(["religion", "radio", "tv"], axis=1)
" + ".join(test_data.columns.tolist())

'ceb + age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + heduc_noans + idlnchld_noans + usemeth_noans'

In [312]:
linear_statmodel_2 = smf.ols("ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle \
                             + nevermarr + heduc_noans + idlnchld_noans + usemeth_noans", test_data)
fitted_model_2 = linear_statmodel_2.fit()
fitted_model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Sun, 09 Dec 2018   Prob (F-statistic):               0.00
Time:                        20:20:18   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.393      0.000      -1.459      -0.681
age                0.1702      0.003     52.271      0.000       0.164       0.177
educ              -0.0729      0.007    -10.285      0.000      -0.087      -0.059
idlnchld           0.0770      0.011      7.042      0.000       0.056       0.098
knowmeth           0.5610      0.121      4.628      0.000       0.323       0.799
usemeth            0.6516      0.048     13.537      0.000       0.557       0.746
agefm             -0.0606      0.007     -9.240      0.000      -0.073      -0.048
heduc             -0.0573      0.008     -7.186      0.000      -0.073      -0.042
urban             -0.2190      0.047     -4.682      0.000      -0.311      -0.127
electric          -0.3207      0.070     -4.584      0.000      -0.458      -0.184
bicycle            0.2046      0.049      4.154      0.000       0.108       0.301
nevermarr         -1.3648      0.189     -7.215      0.000      -1.736      -0.994
heduc_noans       -0.8853      0.145     -6.122      0.000      -1.169      -0.602
idlnchld_noans     0.6565      0.152      4.310      0.000       0.358       0.955
usemeth_noans      0.7732      0.196      3.955      0.000       0.390       1.156
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         357.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [313]:
fitted_model.compare_f_test(fitted_model_2)

(0.9192357784631671, 0.467230554727435, 5.0)

In [314]:
fitted_model.compare_f_test(fitted_model_2)

(0.9192357784631671, 0.467230554727435, 5.0)

pvalue ~ 0.47, model after removal of 3 features dosen't have better results.

In [316]:
round(fitted_model.compare_f_test(fitted_model_2)[1],4)

0.4672

In [317]:
# second model no better than first, so test_data is no longer needed
del test_data

In [319]:
test_data = sub_data.drop(["usemeth", "usemeth_noans"], axis=1)
" + ".join(test_data.columns.tolist())

'ceb + age + educ + religion + idlnchld + knowmeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + heduc_noans + idlnchld_noans'

In [320]:
fitted_model_3 = smf.ols("ceb ~ age + educ + religion + idlnchld + knowmeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + heduc_noans + idlnchld_noans", test_data).fit()
fitted_model_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     432.2
Date:                Sun, 09 Dec 2018   Prob (F-statistic):               0.00
Time:                        20:21:26   Log-Likelihood:                -7822.1
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4330   BIC:                         1.579e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.1314      0.217     -5.225      0.000      -1.556      -0.707
religion[T.other]         -0.1205      0.085     -1.426      0.154      -0.286       0.045
religion[T.protestant]    -0.0312      0.084     -0.372      0.710      -0.196       0.133
religion[T.spirit]        -0.0387      0.079     -0.490      0.624      -0.193       0.116
age                        0.1776      0.003     53.935      0.000       0.171       0.184
educ                      -0.0563      0.007     -7.603      0.000      -0.071      -0.042
idlnchld                   0.0693      0.011      6.191      0.000       0.047       0.091
knowmeth                   0.8622      0.122      7.089      0.000       0.624       1.101
agefm                     -0.0646      0.007     -9.672      0.000      -0.078      -0.052
heduc                     -0.0496      0.008     -6.032      0.000      -0.066      -0.033
urban                     -0.1830      0.048     -3.807      0.000      -0.277      -0.089
electric                  -0.2540      0.079     -3.225      0.001      -0.408      -0.100
radio                     -0.0021      0.052     -0.040      0.968      -0.104       0.100
tv                        -0.1895      0.095     -2.004      0.045      -0.375      -0.004
bicycle                    0.2049      0.051      4.000      0.000       0.104       0.305
nevermarr                 -1.5680      0.193     -8.143      0.000      -1.946      -1.190
heduc_noans               -0.7796      0.148     -5.282      0.000      -1.069      -0.490
idlnchld_noans             0.5265      0.155      3.387      0.001       0.222       0.831
==============================================================================
Omnibus:                      251.332   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              940.025
Skew:                          -0.159   Prob(JB):                    7.52e-205
Kurtosis:                       5.255   Cond. No.                         369.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [321]:
fitted_model.compare_f_test(fitted_model_3)

(91.36702105075616, 1.363410090135935e-39, 2.0)

In [323]:
fitted_model_final = smf.ols("ceb ~ age + educ + religion + idlnchld + knowmeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr", test_data).fit(cov_type="HC1")
fitted_model_final.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     314.3
Date:                Sun, 09 Dec 2018   Prob (F-statistic):               0.00
Time:                        20:22:02   Log-Likelihood:                -7842.2
No. Observations:                4348   AIC:                         1.572e+04
Df Residuals:                    4332   BIC:                         1.582e+04
Df Model:                          15                                         
Covariance Type:                  HC1                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0987      0.261     -4.207      0.000      -1.611      -0.587
religion[T.other]         -0.1193      0.079     -1.511      0.131      -0.274       0.035
religion[T.protestant]    -0.0274      0.079     -0.349      0.727      -0.181       0.127
religion[T.spirit]        -0.0344      0.072     -0.476      0.634      -0.176       0.107
age                        0.1788      0.004     41.305      0.000       0.170       0.187
educ                      -0.0618      0.007     -8.351      0.000      -0.076      -0.047
idlnchld                   0.0549      0.014      3.939      0.000       0.028       0.082
knowmeth                   0.8237      0.172      4.800      0.000       0.487       1.160
agefm                     -0.0655      0.010     -6.567      0.000      -0.085      -0.046
heduc                     -0.0347      0.009     -3.983      0.000      -0.052      -0.018
urban                     -0.1955      0.047     -4.156      0.000      -0.288      -0.103
electric                  -0.2752      0.076     -3.619      0.000      -0.424      -0.126
radio                     -0.0084      0.054     -0.154      0.877      -0.115       0.098
tv                        -0.2119      0.084     -2.515      0.012      -0.377      -0.047
bicycle                    0.2063      0.050      4.160      0.000       0.109       0.303
nevermarr                 -2.2421      0.205    -10.960      0.000      -2.643      -1.841
==============================================================================
Omnibus:                      243.648   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              917.277
Skew:                          -0.134   Prob(JB):                    6.54e-200
Kurtosis:                       5.234   Cond. No.                         354.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""